In [1]:
%%writefile mapper_total.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%d" % (1)

Writing mapper_total.py


In [2]:
%%writefile reducer_total.py
#!/usr/bin/env python2
import sys

total = 0

for line in sys.stdin:
    total += 1

print "%s\t%d" % ('total', total)

Writing reducer_total.py


In [3]:
%%writefile mapper_freq.py
#!/usr/bin/env python2
import sys
import re

def isWordName(word):
    if word[0].isdigit() or (not word[0].isupper()):
        return False
    else:
        for c in word[1:]:
            if c.isupper():
                return False
        return True

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if isWordName(word):
            print "%s\t%d" % (word.lower(), 1)

Writing mapper_freq.py


In [4]:
%%writefile reducer_freq.py
#!/usr/bin/env python2
import sys

current_key = None
word_sum = 0
total = 0

with open('part-00000') as f:
    tmp = f.readline()
total = int(tmp.strip().split('\t', 1)[1])

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            freq = (word_sum / (total * 1.0)) * 100
            print "%s\t%f" % (current_key, freq)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    freq = (word_sum / (total * 1.0)) * 100
    print "%s\t%f" % (current_key, freq)

Writing reducer_freq.py


In [8]:
! cat part-00000

total	11937375


# Job runner

In [5]:
%%bash
TOTAL_DIR="total"
FREQ_DIR="freq"
OUT_DIR="wordcount_stopwords_result"

hdfs dfs -rm -r -skipTrash ${TOTAL_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${FREQ_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Total_count" \
    -D mapreduce.job.reduces=1 \
    -files mapper_total.py,reducer_total.py \
    -mapper "python mapper_total.py" \
    -reducer "python reducer_total.py" \
    -input /data/wiki/en_articles_part \
    -output ${TOTAL_DIR} > /dev/null

hdfs dfs -copyToLocal ./total/part-00000 .

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Frequencies_of_names" \
    -D mapreduce.job.reduces=1 \
    -files mapper_freq.py,reducer_freq.py,part-00000 \
    -mapper "python mapper_freq.py" \
    -reducer "python reducer_freq.py" \
    -input /data/wiki/en_articles_part \
    -output ${FREQ_DIR} > /dev/null    

hdfs dfs -cat ${OUT_DIR}/part-0000* | head  

rm: `total': No such file or directory
rm: `freq': No such file or directory
rm: `wordcount_stopwords_result': No such file or directory
17/11/13 05:44:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/13 05:44:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/13 05:44:27 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/13 05:44:28 INFO mapreduce.JobSubmitter: number of splits:2
17/11/13 05:44:28 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/13 05:44:28 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1510550666160_0001
17/11/13 05:44:31 INFO impl.YarnClientImpl: Submitted application application_1510550666160_0001
17/11/13 05:44:31 INFO mapreduce.Job: The url to track the job: http://668035983b49:8088/proxy/application_1510550666160_0001/
17/11/13 05:44:31 INFO mapreduce.Job: Running job: job_1510550666160_0001
17/11/13 05:44:56 INFO mapreduce.Job: